In [30]:
# Full Python Script
# Python 3.8.8
# Pandas 1.2.4
# Numpy 1.20.1
# Carson Broeker, 10/19/2021
#Test script for converting Mutect, SomaticSniper, and Varscan SNV vcf data to a list of genes
# First, import necessary modules
import math
import pandas as pd
import numpy as np
import sys
import csv
from functools import reduce

# Make it so really large files don't cause the system to error out
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)
# Next, read in all of the VCF files using a for loop
# This step also gets rid of superfluous columns for downstream analyses
# Modify the List elements to suit your project needs
# Make sure all of your vcf files are in the same directory/location as this python script
#dfFVB_Var = pd.read_csv("FVB_NJVarscanSNVs.vcf.snp", sep='\t', usecols=[0,1,2,3,12,14], engine = 'c', dtype=object)
#dfFVB_Var = dfFVB_Var[(dfFVB_Var['somatic_status'] == 'Somatic')]
#dfFVB_Var['#CHROM'] = dfFVB_Var['chrom']
#dfFVB_Var['POS'] = dfFVB_Var['position']
#dfFVB_Var['REF'] = dfFVB_Var['ref']
#dfFVB_Var['ALT'] = dfFVB_Var['var']

#dfFVB_Som = pd.read_csv("FVB_NJSomaticSniperCalls.vcf", sep='\t', skiprows=17, usecols=[0,1,3,4], engine = 'c', dtype=object)

df_Sanger_FVB_Germ = pd.read_csv("FVB_NJ.mgp.v5.snps.dbSNP142.vcf", sep='\t', skiprows=68, usecols=[0,1,3,4], engine='c', dtype=object)

df_Haplotype = pd.read_csv("FVB_NJ_HaplotypeCaller.vcf", sep='\t', skiprows=93, usecols=[0,1,3,4], engine='c', dtype=object)

Myc_List = ['TA39_927','TA41_1938','WT13_842','WT13_864','WT13_1356','WT13_1445','WT13_1576','WT21_222','WT21_1066']
#Myc_List = ['WT13_1356']
for base in Myc_List:
    Varscan_base = pd.read_csv(base+"VarscanSNVs.vcf.snp", sep='\t', usecols=[0,1,2,3,12,14], \
                             engine = 'c', dtype=object)
# Must be loaded as engine='c' as the 'python' engine isn't natively equipped with the necessary modules
# Next, make each dataframe something easier to read and obvious
# If unsure how to modify data, look at the python pandas documentation
    df1 = Varscan_base
# Use masking to include only columns that have deletions or duplications, no inversions or translocations
# Check CNV_Filtering python file for more advanced documentation
    df1 = df1[(df1['somatic_status'] == 'Somatic')]
    df1['#CHROM'] = df1['chrom']
    df1['POS'] = df1['position']
    df1['REF'] = df1['ref']
    df1['ALT'] = df1['var']
    df1['somatic_p_value'] = df1['somatic_p_value'].astype(float)
    #df1FVB_Var = pd.merge(df1, dfFVB_Var, on=['#CHROM','POS','REF','ALT'])
    #df1new = pd.concat([df1, df1FVB_Var]).drop_duplicates(subset=['#CHROM','POS','REF','ALT'], keep=False)
    
    Mutect_base = pd.read_csv(base+"Mutect2Annotated.vcf", sep='\t', skiprows=106, usecols=[0,1,3,4,7], \
                             engine = 'c', dtype=object)
    df2 = Mutect_base
    #df2 = df2[df2['INFO'].str.contains('HIGH|MODERATE|LOW')]
    #df2['READ_DEPTH'] = df2['INFO'].str.extract('DP=(\d+);').astype(int)
    
    #SomaticSniper_base = pd.read_csv(base+"_MycSomaticSniperCallsAnnotated.vcf", sep='\t', skiprows=22, usecols=[0,1,3,4,7], engine = 'c', dtype=object)
    #df3 = SomaticSniper_base
    #df3 = df3[df3['INFO'].str.contains('HIGH|MODERATE|LOW')]
    #df3FVB_Som = pd.merge(df3, dfFVB_Som, on=['#CHROM','POS','REF','ALT'])
    #df3new = pd.concat([df3, df3FVB_Som]).drop_duplicates(subset=['#CHROM','POS','REF','ALT'], keep=False)
    
    df4 = pd.merge(df1, df2, on=['#CHROM','POS','REF','ALT'])
    df5 = pd.merge(df4, df_Sanger_FVB_Germ, on=['#CHROM','POS','REF','ALT'])
    df6 = pd.concat([df4, df5]).drop_duplicates(subset=['#CHROM','POS','REF','ALT'], keep=False)
    df7 = pd.merge(df6, df_Haplotype, on=['#CHROM','POS','REF','ALT'])
    df = pd.concat([df6, df7]).drop_duplicates(subset=['#CHROM','POS','REF','ALT'], keep=False)
    df.dropna(axis='columns', inplace=True)
    #df = pd.merge(df3new, df4, on=['#CHROM','POS','REF','ALT'])
    df['TYPE'] = df['INFO'].str.extract('ANN=.*?\|(.*?)\|')
    df['SCORE'] = df['INFO'].str.extract('ANN=.*?\|.*?\|(.*?)\|')
    df['GENE'] = df['INFO'].str.extract('ANN=.*?\|.*?\|.*?\|(.*?)\|')
    df['MM#CHROM'] = 'mm' + df['#CHROM'].astype(str)
    df['CIRCOS_COLOR'] = 'color=color'+df['SCORE']
    #df = df[(df['GENE'] != 'Erbb2')]
    df = df.sort_values(by=['somatic_p_value'], ascending=True)
    df = df[(df['somatic_p_value'] <= 0.05)]
    #df['RANDOM_NUMBER'] = np.random.randint(0,1000,size=(len(df),1))
    df['COLOR'] = "color=color" + df['REF'] + "to" + df['ALT']
    #df.dropna(axis='columns', inplace=True)
    df = df.sort_values(by=["chrom","position"])
    df["position"] = df["position"].astype(float)
    df['Intermutation_Distance'] = abs(df['position'] - df['position'].shift(1))
    df['Intermutation_Distance_log2'] = np.log2(df['Intermutation_Distance'])
    df = df[(df['Intermutation_Distance_log2'] < 24)]
    df = df[(df['MM#CHROM'] != 'mmY')]
    with pd.ExcelWriter('All_Myc_'+base+'_SNV.xlsx') as writer:
        df.to_excel(writer, sheet_name='Total_SNV_Info')
        df.to_excel(writer, sheet_name='Circos', columns=['MM#CHROM','POS','POS','Intermutation_Distance_log2','COLOR'])
    
    
    df_SNV = pd.read_excel('All_Myc_'+base+'_SNV.xlsx', sheet_name="Circos", usecols=[1,2,3,4,5])
    df_SNV.to_csv('All_'+base+'_Myc_SNV.txt', header=False, index=False, sep='\t', mode='a')
    
    df['Chromosome'] = df['#CHROM']
    df['Position'] = df['POS']
    df['Original Base'] = df['REF']
    df['Alternative Base'] = df['ALT']
    df['Gene'] = df['GENE']
    df['Effect'] = df['TYPE']
    df['Predicted Impact'] = df['SCORE']
    
    with pd.ExcelWriter('All_Myc_'+base+'_SNVs_Pub.xlsx') as writer:
        df.to_excel(writer, sheet_name='Total Info', columns=['Chromosome','Position','Original Base','Alternative Base'\
                                                             ,'Gene','Effect','Predicted Impact'])
    
    
    df = df[df['INFO'].str.contains('HIGH|MODERATE|LOW')]
    
    with pd.ExcelWriter('Impactful_Myc_'+base+'_SNV.xlsx') as writer:
        df.to_excel(writer, sheet_name='Total_SNV_Info')
        df.to_excel(writer, sheet_name='Circos', columns=['MM#CHROM','POS','POS','CIRCOS_COLOR'])
    
    with pd.ExcelWriter('Impactful_Myc_'+base+'_SNVs_Pub.xlsx') as writer:
        df.to_excel(writer, sheet_name='Total Info', columns=['Chromosome','Position','Original Base','Alternative Base'\
                                                             ,'Gene','Effect','Predicted Impact'])
    
    SNV_baseList1 = []
    SNV_baseList1 = df['GENE']
    SNV_baseList1.drop_duplicates(keep='first', inplace=True)
    SNV_baseList1.to_csv('Myc_'+base+'_SNV_Gene_List.csv', sep='\t')
    
from collections import Counter
total_gene_list = []
total_gene_list.clear()
Myc_List = ['TA39_927','TA41_1938','WT13_842','WT13_864','WT13_1356','WT13_1445','WT13_1576','WT21_222','WT21_1066']
#Myc_List = ['3790']
for base in Myc_List:
# Read in csv files to a dataframe
    dfbase = pd.read_csv('Myc_'+base+'_SNV_Gene_List.csv', sep='\t', usecols=[1], header=None)
    base_gene_list = dfbase.values.tolist()
    total_gene_list.append(base_gene_list)
    flat_total_gene_list = [item for sublist in total_gene_list for item in sublist]
    final_flat_total_gene_list = [item for sublist in flat_total_gene_list for item in sublist]
albert = Counter(final_flat_total_gene_list)
with open('Myc_SNV_Total_Gene_Count.csv','w') as csvfile:
    fieldnames=['Gene', 'Count']
    writer=csv.writer(csvfile)
    writer.writerow(fieldnames)
    for key, value in albert.items():
        writer.writerow((key, value))
print('done')

C:\Users\ANDREC~1\AppData\Local\Temp/ipykernel_23240/2927581491.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['#CHROM'] = df1['chrom']
C:\Users\ANDREC~1\AppData\Local\Temp/ipykernel_23240/2927581491.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['POS'] = df1['position']
C:\Users\ANDREC~1\AppData\Local\Temp/ipykernel_23240/2927581491.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



done


In [19]:
#SigProfileExtractor preprocessing
# Full Python Script
# Python 3.8.8
# Pandas 1.2.4
# Numpy 1.20.1
# Carson Broeker, 10/19/2021
#Test script for converting Mutect, SomaticSniper, and Varscan SNV vcf data to a list of genes
# First, import necessary modules
import math
import pandas as pd
import numpy as np
import sys
import csv
from functools import reduce

# Make it so really large files don't cause the system to error out
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)
# Next, read in all of the VCF files using a for loop
# This step also gets rid of superfluous columns for downstream analyses
# Modify the List elements to suit your project needs
# Make sure all of your vcf files are in the same directory/location as this python script
#dfFVB_Var = pd.read_csv("FVB_NJVarscanSNVs.vcf.snp", sep='\t', usecols=[0,1,2,3,12,14], engine = 'c', dtype=object)
#dfFVB_Var = dfFVB_Var[(dfFVB_Var['somatic_status'] == 'Somatic')]
#dfFVB_Var['#CHROM'] = dfFVB_Var['chrom']
#dfFVB_Var['POS'] = dfFVB_Var['position']
#dfFVB_Var['REF'] = dfFVB_Var['ref']
#dfFVB_Var['ALT'] = dfFVB_Var['var']

#dfFVB_Som = pd.read_csv("FVB_NJSomaticSniperCalls.vcf", sep='\t', skiprows=17, usecols=[0,1,3,4], engine = 'c', dtype=object)

df_Sanger_FVB_Germ = pd.read_csv("FVB_NJ.mgp.v5.snps.dbSNP142.vcf", sep='\t', skiprows=68, usecols=[0,1,3,4], engine='c', dtype=object)

df_Haplotype = pd.read_csv("FVB_NJ_HaplotypeCaller.vcf", sep='\t', skiprows=93, usecols=[0,1,3,4], engine='c', dtype=object)

Myc_List = ['TA39_927','TA41_1938','WT13_842','WT13_864','WT13_1356','WT13_1445','WT13_1576','WT21_222','WT21_1066']
#Myc_List = ['WT13_1356']
for base in Myc_List:
    Varscan_base = pd.read_csv(base+"VarscanSNVs.vcf.snp", sep='\t', usecols=[0,1,2,3,12,14], \
                             engine = 'c', dtype=object)
# Must be loaded as engine='c' as the 'python' engine isn't natively equipped with the necessary modules
# Next, make each dataframe something easier to read and obvious
# If unsure how to modify data, look at the python pandas documentation
    df1 = Varscan_base
# Use masking to include only columns that have deletions or duplications, no inversions or translocations
# Check CNV_Filtering python file for more advanced documentation
    df1 = df1[(df1['somatic_status'] == 'Somatic')]
    df1['#CHROM'] = df1['chrom']
    df1['POS'] = df1['position']
    df1['REF'] = df1['ref']
    df1['ALT'] = df1['var']
    df1['somatic_p_value'] = df1['somatic_p_value'].astype(float)
    #df1FVB_Var = pd.merge(df1, dfFVB_Var, on=['#CHROM','POS','REF','ALT'])
    #df1new = pd.concat([df1, df1FVB_Var]).drop_duplicates(subset=['#CHROM','POS','REF','ALT'], keep=False)
    
    Mutect_base = pd.read_csv(base+"Mutect2Annotated.vcf", sep='\t', skiprows=106, usecols=[0,1,3,4,7], \
                             engine = 'c', dtype=object)
    df2 = Mutect_base
    #df2 = df2[df2['INFO'].str.contains('HIGH|MODERATE|LOW')]
    #df2['READ_DEPTH'] = df2['INFO'].str.extract('DP=(\d+);').astype(int)
    
    #SomaticSniper_base = pd.read_csv(base+"_MycSomaticSniperCallsAnnotated.vcf", sep='\t', skiprows=22, usecols=[0,1,3,4,7], engine = 'c', dtype=object)
    #df3 = SomaticSniper_base
    #df3 = df3[df3['INFO'].str.contains('HIGH|MODERATE|LOW')]
    #df3FVB_Som = pd.merge(df3, dfFVB_Som, on=['#CHROM','POS','REF','ALT'])
    #df3new = pd.concat([df3, df3FVB_Som]).drop_duplicates(subset=['#CHROM','POS','REF','ALT'], keep=False)
    
    df4 = pd.merge(df1, df2, on=['#CHROM','POS','REF','ALT'])
    df5 = pd.merge(df4, df_Sanger_FVB_Germ, on=['#CHROM','POS','REF','ALT'])
    df6 = pd.concat([df4, df5]).drop_duplicates(subset=['#CHROM','POS','REF','ALT'], keep=False)
    df7 = pd.merge(df6, df_Haplotype, on=['#CHROM','POS','REF','ALT'])
    df = pd.concat([df6, df7]).drop_duplicates(subset=['#CHROM','POS','REF','ALT'], keep=False)
    df.dropna(axis='columns', inplace=True)
    df["CHROM"] = "chr" + df["#CHROM"]
    #df["FILTER"] = "PASS"
    df["SAMPLE"] = base
    #df = pd.merge(df3new, df4, on=['#CHROM','POS','REF','ALT'])
    #df['TYPE'] = df['INFO'].str.extract('ANN=.*?\|(.*?)\|')
    #df['SCORE'] = df['INFO'].str.extract('ANN=.*?\|.*?\|(.*?)\|')
    #df['GENE'] = df['INFO'].str.extract('ANN=.*?\|.*?\|.*?\|(.*?)\|')
    #df['MM#CHROM'] = 'mm' + df['#CHROM'].astype(str)
    #df['CIRCOS_COLOR'] = 'color=color'+df['SCORE']
    #df = df[(df['GENE'] != 'Erbb2')]
    df = df.sort_values(by=['somatic_p_value'], ascending=True)
    df = df[(df['somatic_p_value'] <= 0.05)]
    #df['RANDOM_NUMBER'] = np.random.randint(0,1000,size=(len(df),1))
    #df.dropna(axis='columns', inplace=True)
    dfnew = df[["SAMPLE","CHROM","POS","REF","ALT"]].copy()
    some_values = ["chr1","chr2","chr3","chr4","chr5","chr6","chr7","chr8","chr9",\
              "chr10","chr11","chr12","chr13","chr14","chr15","chr16","chr17",\
              "chr18","chr19","chrX"]
    dfnew = dfnew.loc[df['CHROM'].isin(some_values)]
    dfnew = dfnew.sort_values(by=["CHROM","POS"])
    dfnew.to_csv(base+"SigProfileExtractor.vcf", sep='\t', index=False)

C:\Users\ANDREC~1\AppData\Local\Temp/ipykernel_26476/3511692749.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['#CHROM'] = df1['chrom']
C:\Users\ANDREC~1\AppData\Local\Temp/ipykernel_26476/3511692749.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['POS'] = df1['position']
C:\Users\ANDREC~1\AppData\Local\Temp/ipykernel_26476/3511692749.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [6]:
# For venn diagrams
from collections import Counter
total_gene_list = []
total_gene_list.clear()
Microacinar_List = ['WT13_842','WT13_864','WT13_1576']
Squamous_List = ['TA39_927','WT13_1445','WT21_1066']
EMT_List = ['TA41_1938','WT13_1356','WT21_222']
#Myc_List = ['3790']
total_gene_list.clear()
for base in Microacinar_List:
# Read in csv files to a dataframe
    dfbase = pd.read_csv('Myc_'+base+'_SNV_Gene_List.csv', sep='\t', usecols=[1], header=None)
    base_gene_list = dfbase.values.tolist()
    total_gene_list.append(base_gene_list)
    flat_total_gene_list = [item for sublist in total_gene_list for item in sublist]
    final_flat_total_gene_list = [item for sublist in flat_total_gene_list for item in sublist]
albert = Counter(final_flat_total_gene_list)
with open('Myc_Microacinar_SNV_Total_Gene_Count.csv','w') as csvfile:
    fieldnames=['Gene', 'Count']
    writer=csv.writer(csvfile)
    writer.writerow(fieldnames)
    for key, value in albert.items():
        writer.writerow((key, value))
total_gene_list.clear()
for base in Squamous_List:
# Read in csv files to a dataframe
    dfbase = pd.read_csv('Myc_'+base+'_SNV_Gene_List.csv', sep='\t', usecols=[1], header=None)
    base_gene_list = dfbase.values.tolist()
    total_gene_list.append(base_gene_list)
    flat_total_gene_list = [item for sublist in total_gene_list for item in sublist]
    final_flat_total_gene_list = [item for sublist in flat_total_gene_list for item in sublist]
albert = Counter(final_flat_total_gene_list)
with open('Myc_Squamous_SNV_Total_Gene_Count.csv','w') as csvfile:
    fieldnames=['Gene', 'Count']
    writer=csv.writer(csvfile)
    writer.writerow(fieldnames)
    for key, value in albert.items():
        writer.writerow((key, value))
total_gene_list.clear()
for base in EMT_List:
# Read in csv files to a dataframe
    dfbase = pd.read_csv('Myc_'+base+'_SNV_Gene_List.csv', sep='\t', usecols=[1], header=None)
    base_gene_list = dfbase.values.tolist()
    total_gene_list.append(base_gene_list)
    flat_total_gene_list = [item for sublist in total_gene_list for item in sublist]
    final_flat_total_gene_list = [item for sublist in flat_total_gene_list for item in sublist]
albert = Counter(final_flat_total_gene_list)
with open('Myc_EMT_SNV_Total_Gene_Count.csv','w') as csvfile:
    fieldnames=['Gene', 'Count']
    writer=csv.writer(csvfile)
    writer.writerow(fieldnames)
    for key, value in albert.items():
        writer.writerow((key, value))
print('done')

done


In [27]:
df = df.sort_values(by=["chrom","position"])
print(df["position"].dtype)
df["position"] = df["position"].astype(float)
df['Intermutation_Distance'] = abs(df['position'] - df['position'].shift(1))
df['Intermutation_Distance_log2'] = np.log2(df['Intermutation_Distance'])
df.head(50)

float64


,chrom,position,ref,var,somatic_status,somatic_p_value,#CHROM,POS,REF,ALT,INFO,TYPE,SCORE,GENE,MM#CHROM,CIRCOS_COLOR,RANDOM_NUMBER,COLOR,Intermutation_Distance,Intermutation_Distance_log2
0,1,3005150.0,C,A,Somatic,4.953183e-03,1,3005150,C,A,DP=72;ECNT=1;NLOD=11.73;N_ART_LOD=-1.616e+00;P...,intergenic_region,MODIFIER,CHR_START-Xkr4,mm1,color=colorMODIFIER,184,color=colorCtoA,NaN,NaN
2,1,3233691.0,C,A,Somatic,2.015202e-03,1,3233691,C,A,DP=83;ECNT=3;NLOD=16.25;N_ART_LOD=-1.740e+00;P...,intron_variant,MODIFIER,Xkr4,mm1,color=colorMODIFIER,342,color=colorCtoA,228541.0,17.802093
3,1,3233723.0,A,G,Somatic,1.089930e-02,1,3233723,A,G,DP=85;ECNT=3;NLOD=13.50;N_ART_LOD=-1.682e+00;P...,intron_variant,MODIFIER,Xkr4,mm1,color=colorMODIFIER,965,color=colorAtoG,32.0,5.000000
4,1,3235138.0,T,C,Somatic,6.238887e-03,1,3235138,T,C,DP=76;ECNT=1;NLOD=7.86;N_ART_LOD=-1.443e+00;PO...,intron_variant,MODIFIER,Xkr4,mm1,color=colorMODIFIER,508,color=colorTtoC,1415.0,10.466586
5,1,3322183.0,T,A,Somatic,3.610201e-05,1,3322183,T,A,DP=91;ECNT=1;NLOD=16.21;N_ART_LOD=-1.750e+00;P...,intron_variant,MODIFIER,Xkr4,mm1,color=colorMODIFIER,370,color=colorTtoA,87045.0,16.409474
6,1,3329956.0,A,T,Somatic,2.796281e-03,1,3329956,A,T,DP=77;ECNT=2;NLOD=10.65;N_ART_LOD=-1.594e+00;P...,intron_variant,MODIFIER,Xkr4,mm1,color=colorMODIFIER,456,color=colorAtoT,7773.0,12.924256
9,1,3537677.0,C,T,Somatic,1.887913e-03,1,3537677,C,T,DP=82;ECNT=2;NLOD=13.55;N_ART_LOD=-1.663e+00;P...,intron_variant,MODIFIER,Xkr4,mm1,color=colorMODIFIER,215,color=colorCtoT,207721.0,17.664288
10,1,3537679.0,C,T,Somatic,9.925075e-04,1,3537679,C,T,DP=82;ECNT=2;NLOD=13.55;N_ART_LOD=-1.663e+00;P...,intron_variant,MODIFIER,Xkr4,mm1,color=colorMODIFIER,17,color=colorCtoT,2.0,1.000000
11,1,3537755.0,G,A,Somatic,3.159305e-02,1,3537755,G,A,DP=87;ECNT=4;NLOD=12.94;N_ART_LOD=-1.643e+00;P...,intron_variant,MODIFIER,Xkr4,mm1,color=colorMODIFIER,124,color=colorGtoA,76.0,6.247928
12,1,3537779.0,T,A,Somatic,1.334880e-02,1,3537779,T,A,DP=88;ECNT=4;NLOD=13.66;N_ART_LOD=-1.670e+00;P...,intron_variant,MODIFIER,Xkr4,mm1,color=colorMODIFIER,179,color=colorTtoA,24.0,4.584963
